In [1]:
# 数値計算に使うライブラリ
import numpy as np
import pandas as pd
from scipy import stats

# 統計モデルを推定するライブラリ
import statsmodels.api as sm
import statsmodels.tsa.api as tsa

# グラフを描画するライブラリ
from matplotlib import pylab as plt
import seaborn as sns

sns.set()

In [2]:
# 表示設定
np.set_printoptions(linewidth=60)
pd.set_option("display.width", 80)

from matplotlib.pylab import rcParams

rcParams["font.family"] = "IPAexGothic"
rcParams["figure.figsize"] = 8, 4

In [3]:
# 乱数の種
np.random.seed(1)

# 正規分布に従う乱数の累積和を作成し、ランダムウォーク系列を作る
sim_size = 100
mu = np.cumsum(stats.norm.rvs(loc=0, scale=1, size=sim_size).round(1)) + 30

# 観測値の作成
y = mu + stats.norm.rvs(loc=0, scale=5, size=sim_size).round(1)

# 時系列インデックスの付与
y_ts = pd.Series(y, index=pd.date_range(start="2020-01-01", periods=sim_size, freq="D"))

# 結果の確認
print(y_ts.head(3))

2020-01-01    29.4
2020-01-02    37.1
2020-01-03    32.5
Freq: D, dtype: float64


In [4]:
# 📘 GitHubレビューコメント

# ✅ 概要
# `UnobservedComponents` モデルでローカルレベルモデルを定義する。
# `use_exact_diffuse=True` によって、初期状態が未知の場合でも
# 「拡散的初期化（exact diffuse initialization）」を利用し、
# 厳密な尤度計算が行えるようにする。

# ✅ コード
mod_local_level_fix = tsa.UnobservedComponents(
    y_ts, level="local level", use_exact_diffuse=True
)

# ✅ 数式
# ローカルレベルモデル（状態空間表現）は以下のように表される：
# $$
# \begin{aligned}
# \text{観測方程式: } & y_t = \alpha_t + \varepsilon_t, \quad \varepsilon_t \sim N(0, \sigma_\varepsilon^2) \\
# \text{状態方程式: } & \alpha_t = \alpha_{t-1} + \eta_t, \quad \eta_t \sim N(0, \sigma_\eta^2)
# \end{aligned}
# $$

# ✅ パラメータ設定
# - `level='local level'`：単純なローカルレベルモデル（ランダムウォーク＋観測ノイズ）
# - `use_exact_diffuse=True`：
#   拡散的初期化を有効化し、初期状態の分散が非常に大きい場合でも安定に尤度を計算できる。
#   これは状態空間モデルにおける「非情報的事前分布（uninformative prior）」に相当。

# ✅ コメント
# この設定により、カルマンフィルタが「初期値に依存しない形」で始まり、
# 非定常系列でも適切に尤度推定が可能となる。
# Box-Jenkins型ARIMAとの対応関係では、ARIMA(0,1,1) に相当するモデル構造となる。

In [5]:
# 📘 GitHubレビューコメント

# ✅ 概要
# ローカルレベルモデルにおいて、固定パラメータを指定してカルマンフィルタを実行している。
# ここでは「観測誤差分散 = 10」「過程誤差分散 = 1」として指定。

# ✅ コード
# フィルタリングの実行
res_local_level_fix = mod_local_level_fix.filter(pd.Series(np.array([10, 1])))

# フィルタ化推定量（Filtered State Estimates）の取得
np.round(res_local_level_fix.level["filtered"][0:3], 5)

# ✅ 数式
# カルマンフィルタの更新式は以下の通り：
# $$
# \begin{aligned}
# \text{予測ステップ:} \quad
# a_{t|t-1} &= a_{t-1|t-1} \\
# P_{t|t-1} &= P_{t-1|t-1} + \sigma_\eta^2 \\
# \text{更新ステップ:} \quad
# v_t &= y_t - a_{t|t-1} \\
# K_t &= \frac{P_{t|t-1}}{P_{t|t-1} + \sigma_\varepsilon^2} \\
# a_{t|t} &= a_{t|t-1} + K_t v_t \\
# P_{t|t} &= (1 - K_t) P_{t|t-1}
# \end{aligned}
# $$

# ✅ パラメータ設定
# - $\sigma_\varepsilon^2 = 10$（観測ノイズの分散）
# - $\sigma_\eta^2 = 1$（状態ノイズの分散）
#
# 状態の変化（トレンド成分）が小さく、観測ノイズが比較的大きいため、
# フィルタは観測値よりもスムーズな系列を生成する。

# ✅ コメント
# `res_local_level_fix.level["filtered"]` は、
# 各時点での状態変数（トレンド成分）の推定値（フィルタ化推定量）を表す。
# この出力によって、時系列データの「潜在的な水準変化」が平滑化された形で得られる。

array([29.4    , 33.43333, 33.07478])

In [6]:
# 📘 GitHubレビューコメント

# ✅ 概要
# 固定パラメータ（観測誤差分散=10, 過程誤差分散=1）を用いて
# ローカルレベルモデル（Local Level Model）をフィルタリングした後、
# 散漫初期化（Diffuse Initialization）に基づく対数尤度を取得している。

# ✅ コード
# 散漫対数尤度（Diffuse Log-Likelihood）の取得
np.round(res_local_level_fix.llf, 5)

# ✅ 数式
# 対数尤度は、カルマンフィルタで得られた1時点先予測誤差に基づき計算される：
# $$
# \log L = -\frac{1}{2} \sum_{t=1}^{T}
# \left[
# \log(2\pi) + \log(F_t) + \frac{v_t^2}{F_t}
# \right]
# $$
# ただし、
# - $v_t = y_t - a_{t|t-1}$：1時点先予測誤差（innovation）
# - $F_t = P_{t|t-1} + \sigma_\varepsilon^2$：その分散（innovation variance）

# ✅ 散漫初期化 (Diffuse Initialization)
# 初期時点で状態分散が非常に大きい ($P_0 \to \infty$) 場合、
# 観測値から情報を得て初期状態を漸近的に推定する手法。
# これにより、未知の初期水準でも安定した尤度計算が可能となる。

# ✅ コメント
# `res_local_level_fix.llf` は「散漫尤度（Diffuse Likelihood）」に基づく
# モデル適合度を表すスカラー値であり、パラメータ推定やモデル比較（AIC計算など）に利用できる。

-321.88824

In [7]:
# 📘 GitHubレビューコメント

# ✅ 概要
# 定常過程（Deterministic Constant Model）を仮定した状態空間モデルを構築し、
# 平滑化を伴わない単純な定数モデルとしてフィルタリングを実施している。
# このモデルは、観測系列が平均値まわりで変動する定常系列である場合に対応。

# ✅ コード
# データの格納とモデル特定
mod_const = tsa.UnobservedComponents(
    y_ts, level="deterministic constant", use_exact_diffuse=True
)

# フィルタリング
res_const = mod_const.fit()

# フィルタ化推定量（状態推定値）を取得
np.round(res_const.level["filtered"][0:3], 5)

# ✅ 数式表現
# 観測方程式（Observation Equation）：
# $$
# y_t = \mu + \varepsilon_t, \quad \varepsilon_t \sim N(0, \sigma_\varepsilon^2)
# $$

# 状態方程式（State Equation）：
# $$
# \mu_t = \mu
# $$

# すなわち、$\mu$ は時間によって変化せず、一定（定常）である。

# ✅ 対数尤度（Log-Likelihood）
# $$
# \log L = -\frac{1}{2} \sum_{t=1}^{T}
# \left[
# \log(2\pi) + \log(\sigma_\varepsilon^2) +
# \frac{(y_t - \mu)^2}{\sigma_\varepsilon^2}
# \right]
# $$

# ✅ コメント
# - 本モデルは最も単純な「状態空間モデル」であり、
#   観測系列の平均値を1つのパラメータとして推定するのみ。
# - ローカルレベルモデルに比べ、トレンドや変化を許容しない。
# - よって、系列が時間的にほぼ一定または定常的な場合にのみ適している。

RUNNING THE L-BFGS-B CODE

           * * *

Machine precision = 2.220D-16
 N =            1     M =           10

At X0         0 variables are exactly at the bounds

At iterate    0    f=  3.35155D+00    |proj g|=  1.66401D-01

At iterate    5    f=  3.24976D+00    |proj g|=  7.84257D-06

           * * *

Tit   = total number of iterations
Tnf   = total number of function evaluations
Tnint = total number of segments explored during Cauchy searches
Skip  = number of BFGS updates skipped
Nact  = number of active bounds at final generalized Cauchy point
Projg = norm of the final projected gradient
F     = final function value

           * * *

   N    Tit     Tnf  Tnint  Skip  Nact     Projg        F
    1      5      6      1     0     0   7.843D-06   3.250D+00
  F =   3.2497595005029982     

CONVERGENCE: NORM_OF_PROJECTED_GRADIENT_<=_PGTOL            


 This problem is unconstrained.


array([29.4 , 33.25, 33.  ])

In [8]:
# 2時点目までの平均
np.mean(y_ts[0:2])

33.25

In [9]:
# 3時点目までの平均
np.mean(y_ts[0:3])

33.0

In [10]:
print("最新時点の状態", np.round(res_const.level["filtered"][-1], 5))
print("観測値の平均値", np.round(np.mean(y_ts), 5))

最新時点の状態 30.931
観測値の平均値 30.931
